Baseline models
* Regression
* CLassification
    * BlockStackNet4() googlenet-(64,6) - highest validation score 0.4863

In [8]:
import torch.nn as nn
from torchvision import models

## Baseline Models is defined
gogglenet





## The data set file path is defined here.

In [9]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

'''
In this function block, the funciton is defined to do
1. def__init__() : get the image from the trainning data path
2. def __len__() : get the size of hte dataset
3. def __getitem__() : get the ralated image the correct dataset
'''
class BlockStackDataset(Dataset):
  '''
  Return the image and its label. If not access or not exist, raise error.
  '''
  def __init__(self, data_frame, img_dir, transform = None):
    self.data_frame = data_frame
    self.img_dir = img_dir
    self.transform = transform

    if not os.path.exists(self.img_dir):
      raise ValueError(f"File path {self.img_dir} not exisits!")
    if not os.access(self.img_dir, os.R_OK):
      raise ValueError(f"File path {self.img_dir} is not readable!")
  '''
  Return the size of the dataset
  '''
  def __len__(self):
    return len(self.data_frame)

  '''
  Read the image and according to its id, get the related data in the train.csv.

  Then return the corresponding image , stable_height, instability_type dataset provided.
  '''
  def __getitem__(self, idx):
    img_name = os.path.join(self.img_dir, str(self.data_frame.iloc[idx, 0])) # the first column in the train == image name
    image = Image.open(img_name + ".jpg")
    label = self.data_frame.iloc[idx, -1] # stable_height
    instability_type = self.data_frame.iloc[idx,4] # instability_type

    if self.transform:
      image = self.transform(image)

    return image, label, instability_type



## Trainner

In [30]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix,classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm
from datetime import datetime as datatime



In [31]:
'''
The function here helps to create the logs recording the experiment results.
'''
def create_log_dir():
  current_time = datatime.now().strftime("%Y%m%d-%H%M%S")
  # log_dir = f'/content/drive/MyDrive/CV final project/runs/experiment_{current_time}'
  # solution_dir = f'/content/drive/MyDrive/CV final project/trained_models/experiment_{current_time}'

  log_dir = f'./runs/experiment_{current_time}'
  solution_dir = f'./runs/experiment_{current_time}'
  if not os.path.exists(log_dir):
    os.makedirs(log_dir)

  if not os.path.exists(solution_dir):
    os.makedirs(solution_dir)

  return log_dir, solution_dir


In [32]:

# Define models: googlenet

class TunnedBlockStackNet8(nn.Module):
    def __init__(self):
        super(TunnedBlockStackNet8, self).__init__()
        # load the pre-trained model: gogglenet
        self.googlenet = models.googlenet(weights = models.GoogLeNet_Weights.IMAGENET1K_V1)

        num_ftrs = self.googlenet.fc.in_features
        self.googlenet.fc = nn.Identity()

        self.fc = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 6)

        )
    def forward(self, x):
      x = self.googlenet(x)
      x = self.fc(x)
      return x

In [40]:
class ClassificationBlockStackTrainer1:
  '''
  The function here helps to initalize the parameters used in the models and pre-process the image
  '''
  def __init__(self, csv_file, img_dir, model, stratify_column='stable_height', test_size=0.2,
                 batch_size=32, num_epochs=10, learning_rate=0.001 ,random_state=42):
        self.csv_file = csv_file
        self.img_dir = img_dir
        self.stratify_column = stratify_column
        self.test_size = test_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.model = model

        # load the train dataset
        self.data_frame = pd.read_csv(csv_file)

        # split data into train and validation dataset
        self.train_data, self.val_data, self.train_ids, self.valid_ids = self.split_dataset()

        # pre-processing the images
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),

              #ADD MORE TRANSFORM METHODS HERE

            transforms.RandomHorizontalFlip(),## ADDDED
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)), ## ADDDED
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),## ADDDED
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


        self.train_loader = self.create_dataloader(self.train_data, self.transform)
        self.val_loader = self.create_dataloader(self.val_data, self.transform, shuffle=False)


        # use the gpu
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # set the loss weight
        '''
        Changed some thing here : to see the results
        1. add the class_weights to each class labels with different weights,
           more lables occurece with lower weight, few label occurence with higher weight;
           without the weights, all the labels will be treated equally.

        2. used to apply L2 regularization (also called weight decay).
        The primary purpose of weight decay is to prevent overfitting by penalizing large weights.
         It adds a penalty to the loss function based on the size of the weights
         helps it generalize better to unseen data.
         Lnew = Lold + weight_decay * sum(weight^2)

        3. add scheduler to Reduces the learning rate after every step_size epochs.
          After every 4 epochs, the learning rate will be multiplied by gamma (0.1 here), reducing it by 90%.
        '''
        class_weights = torch.tensor([100/25 , 100/25, 100/20,  100/15, 100/10,  100/5], device= self.device)
        self.criterion = nn.CrossEntropyLoss(weight=class_weights) # CrossEntropy for multi categorical-label predication
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate , weight_decay=0.0001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size = 15, gamma=0.1)


  '''
  The function helps to split the data set into the training and validation dataset according to the
  size pre-determined.
  '''

  def split_dataset(self):
    split = StratifiedShuffleSplit(n_splits=1, test_size=self.test_size, random_state=42)
    train_ids = []
    valid_ids = []
    for train_idx, val_idx in split.split(self.data_frame, self.data_frame[self.stratify_column]):
      train_data = self.data_frame.iloc[train_idx]
      val_data = self.data_frame.iloc[val_idx]
      train_ids.append(train_idx)
      valid_ids.append(val_idx)
    print(f"Train dataset size: {len(train_data)}",
       f"Validation dataset size: {len(val_data)}",
       f"length of train_ids{(len(train_ids))}",
       f"length of valid_ids{(len(valid_ids))}")
    return train_data, val_data, train_ids, valid_ids


  '''
  The function helps to loda the image
  '''
  def create_dataloader(self, data_frame, transform, shuffle=True):
    dataset = BlockStackDataset(data_frame, self.img_dir, transform=transform) # transform 可以用来数据增强
    return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)


  def generate_classification_report(self, outputs, labels):
    predicted = outputs # 'outputs' is already a numpy array after prediction
    labels = labels
    print(classification_report(labels, predicted, zero_division=0))



  def calculate_confusion_matrix(self, outputs, labels):
    predicted = outputs
    labels = labels
    matrix = confusion_matrix(labels, predicted)
    print(matrix)


  def validate(self):
    self.model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_labels = []  # add all the lables
    all_predictions = []  # add all the prediction

    with torch.no_grad():
        for inputs, labels, _ in self.val_loader:
            inputs, labels = inputs.to(self.device), labels.to(self.device).long()
            labels = labels - 1
            outputs = self.model(inputs)
            predicted = torch.argmax(outputs, 1)

            # collecting all the lables and predictions
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            # calculate the loss
            loss = self.criterion(outputs, labels)
            val_loss += loss.item()

            # calculate the correct predication
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    # calculate the accuracy rate
    val_accuracy = correct_predictions / total_samples
    self.generate_classification_report(np.array(all_predictions), np.array(all_labels))
    self.calculate_confusion_matrix(np.array(all_predictions), np.array(all_labels))
    return val_loss/len(self.val_loader),val_accuracy, all_labels, all_predictions


  '''
  The function here is used as the main training function on the image by using the pre-definned models in
  hte first model class.
  '''

  def train(self):
      _, solution_dir = create_log_dir()

      best_val_accuracy = 0.0
      for epoch in range(self.num_epochs):
          self.model.train()
          running_loss = 0.0
          running_accuracy = 0.0

          # monitor the process
          with tqdm(self.train_loader, unit="batch") as tepoch:
              tepoch.set_description(f"Epoch {epoch + 1}/{self.num_epochs}")

              for inputs, labels, _ in tepoch:
                  inputs, labels = inputs.to(self.device), labels.to(self.device).long()
                  labels = labels - 1

                  #forward propagation
                  self.optimizer.zero_grad()
                  raw_outputs = self.model(inputs)
                  loss = self.criterion(raw_outputs, labels)  # loss calculation

                  loss.backward()  # backward propagation
                  self.optimizer.step()

                  #Loss calculating
                  running_loss += loss.item()
                  _, predicted = torch.max(raw_outputs, 1)
                  accuracy = (predicted == labels).sum().item()/ labels.size(0)
                  running_accuracy += accuracy
                  tepoch.set_postfix(loss=running_loss / len(self.train_loader),
                            accuracy=running_accuracy / len(self.train_loader))



          self.scheduler.step()
          print(self.scheduler.get_last_lr())


          val_loss,val_accuracy, all_labels, all_predictions = self.validate()
          print(f"Epoch {epoch + 1}/{self.num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

          if val_accuracy > best_val_accuracy:
              best_val_accuracy = val_accuracy
              torch.save(self.model.state_dict(), f'{solution_dir}/best_model.pth')
              print('Best model saved!')

      print('Finished Training')
      print(f'Best validation accuracy: {best_val_accuracy:.4f}')



In [39]:
# test
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=1,
        batch_size=32
        )
    trainer.train()

AttributeError: cannot assign module before Module.__init__() call

In [65]:
train_ids = trainer.train_ids
valid_ids = trainer.valid_ids

In [1]:
len(valid_ids[0])

NameError: name 'valid_ids' is not defined

In [67]:
val_loss,val_accuracy, all_labels, all_predictions = trainer.validate()

              precision    recall  f1-score   support

           0       0.74      0.70      0.72       384
           1       0.70      0.70      0.70       384
           2       0.61      0.68      0.64       307
           3       0.58      0.62      0.60       230
           4       0.53      0.49      0.51       154
           5       0.43      0.35      0.39        77

    accuracy                           0.64      1536
   macro avg       0.60      0.59      0.59      1536
weighted avg       0.64      0.64      0.64      1536

[[267  41  32  26  11   7]
 [ 34 269  38  27  12   4]
 [ 20  35 209  23  12   8]
 [ 19  13  31 143  18   6]
 [ 14  10  20  24  75  11]
 [  7  14  13   3  13  27]]


In [68]:
comparison = map(lambda all_labels, all_predictions: all_labels == all_predictions, all_labels, all_predictions)
sum(comparison)/len(all_labels)

0.64453125

In [70]:
val_data = trainer.val_data
val_data.head()

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height
7334,956915,2,2,4,2,1,1
3955,516709,1,2,5,1,1,3
619,77447,1,2,5,1,1,3
1594,212770,2,2,4,2,1,1
5645,745098,2,2,6,1,1,4


In [80]:
val_data['predicted'] = [pred+1 for pred in all_predictions]
val_data.head()

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_39098/1482792748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['predicted'] = [pred+1 for pred in all_predictions]


,id,shapeset,type,total_height,instability_type,cam_angle,stable_height,predicted
7334,956915,2,2,4,2,1,1,1
3955,516709,1,2,5,1,1,3,2
619,77447,1,2,5,1,1,3,3
1594,212770,2,2,4,2,1,1,1
5645,745098,2,2,6,1,1,4,2


In [84]:
val_data['pred_type'] = val_data['predicted'] == val_data['stable_height']
val_data.head()

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_39098/2673748297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['pred_type'] = val_data['predicted'] == val_data['stable_height']


,id,shapeset,type,total_height,instability_type,cam_angle,stable_height,predicted,pred_type
7334,956915,2,2,4,2,1,1,1,True
3955,516709,1,2,5,1,1,3,2,False
619,77447,1,2,5,1,1,3,3,True
1594,212770,2,2,4,2,1,1,1,True
5645,745098,2,2,6,1,1,4,2,False


In [81]:
len(val_data[val_data['predicted']== val_data['stable_height']])/len(valid_ids[0])

0.64453125

In [88]:
val_data['pred_type'].sum()/len(val_data)

0.64453125

In [91]:
val_data.groupby('instability_type').pred_type.count()

instability_type
0    369
1    777
2    390
Name: pred_type, dtype: int64

In [93]:
val_data.groupby('shapeset').pred_type.sum()/val_data.groupby('shapeset').pred_type.count()

shapeset
1    0.616105
2    0.659681
Name: pred_type, dtype: float64

In [94]:
val_data.groupby('type').pred_type.sum()/val_data.groupby('type').pred_type.count()

type
1    0.768021
2    0.522639
Name: pred_type, dtype: float64

In [92]:
val_data.groupby('instability_type').pred_type.sum()/val_data.groupby('instability_type').pred_type.count()

instability_type
0    0.544715
1    0.550837
2    0.925641
Name: pred_type, dtype: float64

In [3]:
import pandas as pd
df = pd.read_csv('./COMP90086_2024_Project_train/train.csv')
df.head()

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height
0,54,2,1,3,1,1,2
1,173,1,1,4,1,2,1
2,245,1,1,4,1,2,1
3,465,2,1,5,0,1,5
4,611,2,1,3,1,1,1


In [6]:
df.groupby('instability_type').count()

,id,shapeset,type,total_height,cam_angle,stable_height
instability_type,,,,,,
0,1920,1920,1920,1920,1920,1920
1,3840,3840,3840,3840,3840,3840
2,1920,1920,1920,1920,1920,1920


In [10]:
best_model = torch.load('./runs/experiment_20241012-010838/best_model.pth')

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_67928/1284981419.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load('./runs/experiment_20

In [15]:
best_model = torch.load('./runs/experiment_20241012-010838/best_model.pth',map_location=torch.device('cpu'))
best_model.eval()

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_67928/1886415100.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load('./runs/experiment_20

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

# Load Model

In [36]:
class LoadModel(nn.Module):
  '''
  The function here helps to initalize the parameters used in the models and pre-process the image
  '''
  def __init__(self, csv_file, img_dir, model, stratify_column='stable_height', test_size=0.2,
                 batch_size=32, num_epochs=10, learning_rate=0.001 ,random_state=42):
        self.csv_file = csv_file
        self.img_dir = img_dir
        self.stratify_column = stratify_column
        self.test_size = test_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.model = model

        # load the train dataset
        self.data_frame = pd.read_csv(csv_file)

        # split data into train and validation dataset
        self.train_data, self.val_data, self.train_ids, self.valid_ids = self.split_dataset()

        # pre-processing the images
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),

              #ADD MORE TRANSFORM METHODS HERE

            transforms.RandomHorizontalFlip(),## ADDDED
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)), ## ADDDED
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),## ADDDED
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


        self.train_loader = self.create_dataloader(self.train_data, self.transform)
        self.val_loader = self.create_dataloader(self.val_data, self.transform, shuffle=False)


        # use the gpu
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # set the loss weight
        '''
        Changed some thing here : to see the results
        1. add the class_weights to each class labels with different weights,
           more lables occurece with lower weight, few label occurence with higher weight;
           without the weights, all the labels will be treated equally.

        2. used to apply L2 regularization (also called weight decay).
        The primary purpose of weight decay is to prevent overfitting by penalizing large weights.
         It adds a penalty to the loss function based on the size of the weights
         helps it generalize better to unseen data.
         Lnew = Lold + weight_decay * sum(weight^2)

        3. add scheduler to Reduces the learning rate after every step_size epochs.
          After every 4 epochs, the learning rate will be multiplied by gamma (0.1 here), reducing it by 90%.
        '''
        class_weights = torch.tensor([100/25 , 100/25, 100/20,  100/15, 100/10,  100/5], device= self.device)
        self.criterion = nn.CrossEntropyLoss(weight=class_weights) # CrossEntropy for multi categorical-label predication
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate , weight_decay=0.0001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size = 15, gamma=0.1)


  '''
  The function helps to split the data set into the training and validation dataset according to the
  size pre-determined.
  '''

  def split_dataset(self):
    split = StratifiedShuffleSplit(n_splits=1, test_size=self.test_size, random_state=42)
    train_ids = []
    valid_ids = []
    for train_idx, val_idx in split.split(self.data_frame, self.data_frame[self.stratify_column]):
      train_data = self.data_frame.iloc[train_idx]
      val_data = self.data_frame.iloc[val_idx]
      train_ids.append(train_idx)
      valid_ids.append(val_idx)
    print(f"Train dataset size: {len(train_data)}",
       f"Validation dataset size: {len(val_data)}",
       f"length of train_ids{(len(train_ids))}",
       f"length of valid_ids{(len(valid_ids))}")
    return train_data, val_data, train_ids, valid_ids


  '''
  The function helps to loda the image
  '''
  def create_dataloader(self, data_frame, transform, shuffle=True):
    dataset = BlockStackDataset(data_frame, self.img_dir, transform=transform) # transform 可以用来数据增强
    return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)


  def generate_classification_report(self, outputs, labels):
    predicted = outputs # 'outputs' is already a numpy array after prediction
    labels = labels
    print(classification_report(labels, predicted, zero_division=0))



  def calculate_confusion_matrix(self, outputs, labels):
    predicted = outputs
    labels = labels
    matrix = confusion_matrix(labels, predicted)
    print(matrix)


  def validate(self):
    self.model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_labels = []  # add all the lables
    all_predictions = []  # add all the prediction

    with torch.no_grad():
        for inputs, labels, _ in self.val_loader:
            inputs, labels = inputs.to(self.device), labels.to(self.device).long()
            labels = labels - 1
            outputs = self.model(inputs)
            predicted = torch.argmax(outputs, 1)

            # collecting all the lables and predictions
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            # calculate the loss
            loss = self.criterion(outputs, labels)
            val_loss += loss.item()

            # calculate the correct predication
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    # calculate the accuracy rate
    val_accuracy = correct_predictions / total_samples
    self.generate_classification_report(np.array(all_predictions), np.array(all_labels))
    self.calculate_confusion_matrix(np.array(all_predictions), np.array(all_labels))
    return val_loss/len(self.val_loader),val_accuracy, all_labels, all_predictions


  '''
  The function here is used as the main training function on the image by using the pre-definned models in
  hte first model class.
  '''

  def train(self):
      _, solution_dir = create_log_dir()

      best_val_accuracy = 0.0
      for epoch in range(self.num_epochs):
          self.model.train()
          running_loss = 0.0
          running_accuracy = 0.0

          # monitor the process
          with tqdm(self.train_loader, unit="batch") as tepoch:
              tepoch.set_description(f"Epoch {epoch + 1}/{self.num_epochs}")

              for inputs, labels, _ in tepoch:
                  inputs, labels = inputs.to(self.device), labels.to(self.device).long()
                  labels = labels - 1

                  #forward propagation
                  self.optimizer.zero_grad()
                  raw_outputs = self.model(inputs)
                  loss = self.criterion(raw_outputs, labels)  # loss calculation

                  loss.backward()  # backward propagation
                  self.optimizer.step()

                  #Loss calculating
                  running_loss += loss.item()
                  _, predicted = torch.max(raw_outputs, 1)
                  accuracy = (predicted == labels).sum().item()/ labels.size(0)
                  running_accuracy += accuracy
                  tepoch.set_postfix(loss=running_loss / len(self.train_loader),
                            accuracy=running_accuracy / len(self.train_loader))



          self.scheduler.step()
          print(self.scheduler.get_last_lr())


          val_loss,val_accuracy, all_labels, all_predictions = self.validate()
          print(f"Epoch {epoch + 1}/{self.num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

          if val_accuracy > best_val_accuracy:
              best_val_accuracy = val_accuracy
              torch.save(self.model.state_dict(), f'{solution_dir}/best_model.pth')
              print('Best model saved!')

      print('Finished Training')
      print(f'Best validation accuracy: {best_val_accuracy:.4f}')



In [37]:
model = TunnedBlockStackNet8()

bm = LoadModel(
    csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=30,
        batch_size=32)

AttributeError: cannot assign module before Module.__init__() call

In [35]:
# 指定设备（GPU 或 CPU）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bm.to(device)

# 加载保存的 state_dict
state_dict = torch.load('./runs/experiment_20241012-010838/best_model.pth', map_location=device)

# 将 state_dict 加载到模型
bm.load_state_dict(state_dict)

AttributeError: 'LoadModel' object has no attribute 'to'

Test Model

Net8, epoch = 8, no batch_size defined

In [13]:
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        # csv_file = '/content/drive/MyDrive/CV final project/train data/train.csv', ##
        # img_dir='/content/drive/MyDrive/CV final project/train data/train', ##
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=8,
        #batch_size=32
        )
    trainer.train()

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /Users/macbookpro/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:03<00:00, 14.0MB/s]


Train dataset size: 6144 Validation dataset size: 1536


Epoch 1/8: 100%|██████████| 192/192 [06:38<00:00,  2.07s/batch, accuracy=0.274, loss=1.58]   


[0.001]
              precision    recall  f1-score   support

           0       0.49      0.42      0.45       384
           1       0.38      0.45      0.41       384
           2       0.30      0.62      0.41       307
           3       0.25      0.06      0.09       230
           4       0.26      0.12      0.16       154
           5       0.00      0.00      0.00        77

    accuracy                           0.36      1536
   macro avg       0.28      0.28      0.25      1536
weighted avg       0.34      0.36      0.33      1536

[[161 134  78   4   7   0]
 [ 70 172 132   4   6   0]
 [ 26  71 191  12   7   0]
 [ 28  47 124  13  18   0]
 [ 32  12  76  16  18   0]
 [ 13  11  35   4  14   0]]
Epoch 1/8, Validation Loss: 1.5266, Validation Accuracy: 0.3613
Best model saved!


Epoch 2/8: 100%|██████████| 192/192 [06:44<00:00,  2.11s/batch, accuracy=0.363, loss=1.45]   


...

[0.001]
              precision    recall  f1-score   support

           0       0.77      0.38      0.51       384
           1       0.69      0.51      0.58       384
           2       0.53      0.53      0.53       307
           3       0.42      0.47      0.44       230
           4       0.25      0.69      0.37       154
           5       0.12      0.12      0.12        77

    accuracy                           0.47      1536
   macro avg       0.46      0.45      0.43      1536
weighted avg       0.56      0.47      0.49      1536

[[147  52  53  45  80   7]
 [ 23 194  65  32  46  24]
 [  6  17 162  56  51  15]
 [  9   5  19 109  84   4]
 [  6   6   6  17 106  13]
 [  1   8   0   1  58   9]]
Epoch 6/8, Validation Loss: 1.3388, Validation Accuracy: 0.4733


Epoch 7/8: 100%|██████████| 192/192 [06:46<00:00,  2.12s/batch, accuracy=0.49, loss=1.22]    


[0.001]
              precision    recall  f1-score   support

           0       0.44      0.70      0.54       384
           1       0.51      0.58      0.54       384
           2       0.48      0.23      0.31       307
           3       0.42      0.33      0.37       230
           4       0.40      0.38      0.39       154
           5       0.31      0.05      0.09        77

    accuracy                           0.46      1536
   macro avg       0.43      0.38      0.37      1536
weighted avg       0.45      0.46      0.43      1536

[[268  78  14  18   6   0]
 [103 222  22  20  15   2]
 [ 72 101  71  47  14   2]
 [ 66  20  33  76  35   0]
 [ 63   7   5  15  59   5]
 [ 38   9   4   3  19   4]]
Epoch 7/8, Validation Loss: 1.3688, Validation Accuracy: 0.4557


Epoch 8/8: 100%|██████████| 192/192 [06:54<00:00,  2.16s/batch, accuracy=0.515, loss=1.2]    


[0.001]
              precision    recall  f1-score   support

           0       0.64      0.58      0.61       384
           1       0.60      0.60      0.60       384
           2       0.48      0.57      0.52       307
           3       0.47      0.50      0.49       230
           4       0.36      0.36      0.36       154
           5       0.26      0.13      0.17        77

    accuracy                           0.53      1536
   macro avg       0.47      0.46      0.46      1536
weighted avg       0.52      0.53      0.52      1536

[[223  80  40  29  10   2]
 [ 47 231  71  18  14   3]
 [ 21  49 174  41  20   2]
 [ 24   8  48 116  28   6]
 [ 23   7  17  36  55  16]
 [ 12  10  13   7  25  10]]
Epoch 8/8, Validation Loss: 1.1596, Validation Accuracy: 0.5267
Best model saved!
Finished Training
Best validation accuracy: 0.5267
